In [1]:
import warnings

# Filter specific warning
warnings.filterwarnings("ignore")

In [36]:
import numpy as np
import pandas as pd
import random

df_org = pd.read_csv('../Dataset/kl.csv')
df_org = df_org.head(10000)

# Selecting the relevant variables
col = ['Age', 'Overall', 'Potential', 'Wage', 'Special', 'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Height', 'Weight', 'Club', 'Value']
df = df_org[col]

df.columns = ['Age', 'Overall', 'Potential', 'Wage', 'Special', 'PreferredFoot', 'InternationalReputation', 'WeakFoot',
              'SkillMoves', 'Height', 'Weight', 'Club', 'Value']


In [39]:
# Changing the format of several string variabls
val = []
for i in df['Value']:
    if i[-1] == 'M':
        val.append(float(i[1:-1]) * 1000000)
    elif i[-1] == 'K':
        val.append(float(i[1:-1]) * 1000)
    else:
        val.append(0)
df.loc[:, 'Value'] = val

val = []
for i in df['Wage']:
    if i[-1] == 'M':
        val.append(float(i[1:-1]) * 1000000)
    elif i[-1] == 'K':
        val.append(float(i[1:-1]) * 1000)
    else:
        val.append(0)
df.loc[:, 'Wage'] = val

height = df.Height.str.split('\'', 1)
height0 = []
height1 = []
for i in height:
    if type(i) is list:
        height0.append(i[0])
        height1.append(i[1])
    else:
        height0.append(0)
        height1.append(0)

height_cm = []
for i in range(len(height)):
    height_cm.append(round((float(height0[i]) * 30.48 + float(height1[i]) * 2.54) / 100, 3))
df.loc[:, 'Height'] = height_cm

df['Weight'] = pd.to_numeric(df['Weight'].str.rstrip('lbs'))

df = df.iloc[np.where((df['Value'] > 0) & (df['Height'] > 0))]

df.rename({'Club': 'cluster'}, axis=1, inplace=True)
df.rename({'Value': 'y'}, axis=1, inplace=True)

In [40]:
df.head(7)

,Age,Overall,Potential,Wage,Special,PreferredFoot,InternationalReputation,WeakFoot,SkillMoves,Height,Weight,cluster,y
0,31.0,94.0,94,565000.0,2202,Left,5.0,4.0,4.0,1.702,159,FC Barcelona,110500000.0
1,33.0,94.0,94,405000.0,2228,Right,5.0,4.0,5.0,1.880,183,Juventus,77000000.0
2,26.0,92.0,93,290000.0,2143,Right,5.0,5.0,5.0,1.753,150,Paris Saint-Germain,118500000.0
3,27.0,91.0,93,260000.0,1471,Right,4.0,3.0,1.0,1.930,168,Manchester United,72000000.0
4,27.0,91.0,92,355000.0,2281,Right,4.0,5.0,4.0,1.803,154,Manchester City,102000000.0
5,27.0,91.0,91,340000.0,2142,Right,4.0,4.0,4.0,1.727,163,Chelsea,93000000.0
6,32.0,91.0,91,420000.0,2280,Right,4.0,4.0,4.0,1.727,146,Real Madrid,67000000.0


In [41]:
min(df['y'])

40000.0

In [42]:
df['y'] = df['y'] / 10000

In [43]:
df.head(7)

,Age,Overall,Potential,Wage,Special,PreferredFoot,InternationalReputation,WeakFoot,SkillMoves,Height,Weight,cluster,y
0,31.0,94.0,94,565000.0,2202,Left,5.0,4.0,4.0,1.702,159,FC Barcelona,11050.0
1,33.0,94.0,94,405000.0,2228,Right,5.0,4.0,5.0,1.880,183,Juventus,7700.0
2,26.0,92.0,93,290000.0,2143,Right,5.0,5.0,5.0,1.753,150,Paris Saint-Germain,11850.0
3,27.0,91.0,93,260000.0,1471,Right,4.0,3.0,1.0,1.930,168,Manchester United,7200.0
4,27.0,91.0,92,355000.0,2281,Right,4.0,5.0,4.0,1.803,154,Manchester City,10200.0
5,27.0,91.0,91,340000.0,2142,Right,4.0,4.0,4.0,1.727,163,Chelsea,9300.0
6,32.0,91.0,91,420000.0,2280,Right,4.0,4.0,4.0,1.727,146,Real Madrid,6700.0


## The analysis

In [44]:
## Дані будуть розділені на тренувальну і тестувальну вибірки, так щоб не було однакових значень 'cluster' між цими вибірками.
version = 'CV'
depth = 5
NoCov = 2

sample_clusters = random.sample(set(df.cluster.unique()), 20)

df_tr = df.loc[df['cluster'].isin(sample_clusters)]
df_test = df.loc[~df['cluster'].isin(sample_clusters)]

In [ ]:
from RealData import ClusteredRETCO as RETCOfile

RETCO = RETCOfile.CreateTree(version, df_tr, depth=depth, min_leaf_sample=3, StoppingRule='Yes', random_subspace=None)

best overall metric: 117021.22572890949
best overall metric: 116989.71903926111
best overall metric: 116802.08185038944
best overall metric: 116691.3798215242
best overall metric: 115963.262203431
best overall metric: 115842.17187930415
best overall metric: 115046.02908522308
best overall metric: 111200.98249414696
best overall metric: 104578.39382259095
best overall metric: 95728.34455678222
best overall metric: 86081.49783417149
best overall metric: 76439.85991254497
best overall metric: 65442.35944828286
best overall metric: 58783.968434596056
best overall metric: 54465.73247333477


In [ ]:
from RealData import CHRT as RTfile

tree = RTfile.decision_tree_algorithm(version, df_tr, max_depth=depth, min_leaf_sample=3, counter=0,
                                      random_subspace=None)
covariates_pred_test = df_test.iloc[:, :-NoCov].values
col_names = df_test.iloc[:, :-NoCov].columns

In [12]:
prediction_test_RETCO = RETCOfile.predictionFun(col_names, covariates_pred_test, RETCO)
prediction_test = RTfile.decision_tree_predictions(df_test, tree)

In [13]:
y_test = df_test['y'].values

In [24]:
new_y_test = y_test[1:]

In [27]:
RETCOerror = sum((new_y_test - prediction_test_RETCO) ** 2) / len(new_y_test)

In [29]:
RTerror = sum((y_test - prediction_test) ** 2) / len(y_test)

In [31]:
print('RETCOerror:', RETCOerror, '\nRTerror:', RTerror)

RETCOerror: 514139.3510490883 
RTerror: 554992.8971176653
